In [13]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [156]:
df = pd.read_csv("data/100x100.csv")
df_2 = pd.read_csv("data/100x100_2.csv")

In [ ]:
delta=df[['u_id', 'a_id', 'score']]
delta_2=df_2[['u_id', 'a_id', 'score']]

In [16]:
data_matrix

a_id,1,30,32,199,226,227,323,339,356,849,...,34134,34561,34599,34618,34933,35790,35849,37349,37450,38000
u_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,9.0,8.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,10.0,7.0,9.0,7.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
31,10.0,7.0,5.0,0.0,0.0,9.0,0.0,10.0,0.0,0.0,...,0.0,8.0,9.0,0.0,0.0,0.0,6.0,0.0,9.0,0.0
33,9.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,0.0,0.0,0.0,0.0,9.0,10.0,0.0,0.0,8.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5520,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5956,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data_matrix.columns

Index([    1,     5,     6,     7,    15,    16,    19,    20,    21,    22,
       ...
       39417, 39456, 39468, 39491, 39523, 39533, 39539, 39597, 39799, 40004],
      dtype='int64', name='a_id', length=1590)

In [ ]:
user_id_to_index = {} #helper function
for idx in range(len(data_matrix.index)):
    u_id = data_matrix.index[idx]
    user_id_to_index[u_id] = idx
    
user_id_to_index[4808]

93

In [155]:
anime_id_to_index = {} #helper function
for idx in range(len(data_matrix.columns)):
    a_id = data_matrix.columns[idx]
    anime_id_to_index[a_id] = idx

anime_id_to_index[226]

4

In [8]:
user_id_to_index[1892]

999

In [158]:
def baseline_model(x,df):


    data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    delta = df[['u_id', 'a_id', 'score']]
    avg_rating = df['score'].mean()
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    b_u=cp.Variable(num_users)
    b_i=cp.Variable(num_anime)

    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx
        
    R_ui=[]
    for ui in range(len(delta)):
        u = user_id_to_index[int(delta.at[ui,'u_id'])]
        i = anime_id_to_index[int(delta.at[ui,'a_id'])]
        R_ui.append((avg_rating + b_u[u] + b_i[i])) #R_ui=avg_rating+b_u[u]+b_i[i]

    training_error=[]
    for ui in range(len(delta)):
        training_error.append((R_ui[ui]-int(delta.at[ui,'score']))**2) #(R_ui-A_ui)^2

    training_error_sum=cp.sum(training_error) #sum((R_ui-A_ui)^2)

    regularized_u=cp.sum_squares(b_u) #sum(b_u^2)
    regularized_i=cp.sum_squares(b_i) #sum(b_i^2)

    obj=cp.Minimize(training_error_sum+ x*(regularized_u+regularized_i))
    prob=cp.Problem(obj)
    prob.solve()
    print("status:", prob.status)
    
    R = np.zeros((num_users, num_anime))
    for u in range(num_users):
        for i in range(num_anime):
            R[u, i] = avg_rating + b_u.value[u] + b_i.value[i]

    if prob.status == cp.OPTIMAL:
        print("The predicted matrix is ",  np.matrix(R))
    
    return np.matrix(R)



In [161]:
R=baseline_model(5,df_2)

status: optimal
The predicted matrix is  [[7.68199183 8.5954371  8.59176109 ... 7.82774308 8.38352013 7.19850524]
 [7.66556004 8.5790053  8.5753293  ... 7.81131128 8.36708834 7.18207345]
 [7.59622352 8.50966878 8.50599278 ... 7.74197476 8.29775182 7.11273693]
 ...
 [7.76473507 8.67818033 8.67450432 ... 7.91048631 8.46626336 7.28124847]
 [6.28517517 7.19862043 7.19494442 ... 6.43092641 6.98670346 5.80168857]
 [6.68855469 7.60199996 7.59832395 ... 6.83430594 7.39008299 6.2050681 ]]


In [150]:
R[93,4]

np.float64(7.876027329614334)

In [165]:
def RMSE(R,df):
    data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    delta = df[['u_id', 'a_id', 'score']]
    N=len(delta)

    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx

    LSE=[]
    for ui in range(len(delta)):
            u = user_id_to_index[int(delta.at[ui,'u_id'])]
            i = anime_id_to_index[int(delta.at[ui,'a_id'])]
            LSE.append((R[u,i]-int(delta.at[ui,'score']))**2)
    RMSE=np.sqrt(np.sum(LSE)/N)

    return RMSE
    

In [ ]:
RMSE(R,df_2) #baseline model RMSE2, lambda=5

np.float64(1.485386110495448)